In [7]:
import os
import json
import xml.etree.ElementTree as ET
from collections import defaultdict

opj = os.path.join

In [8]:
BASE_DIR = "/home/dmsai2/mmdetection/data/tooth"

In [9]:
TRAIN_DIR = opj(BASE_DIR, "train")
VAL_DIR = opj(BASE_DIR, "val")
ANN_DIR = opj(BASE_DIR, "annotations")

ORIGINAL_DATA_DIR = opj(BASE_DIR, "images")
TRAIN_ORG_ANN_DIR = opj(BASE_DIR, "train_xmls")
VAL_ORG_ANN_DIR = opj(BASE_DIR, "val_xmls")

TRAIN_OUTPUT_FILE = opj(ANN_DIR, "train.json")
VAL_OUTPUT_FILE = opj(ANN_DIR, "val.json")

In [14]:
TEST_DIR = opj(BASE_DIR, "test")
TEST_ORG_ANN_DIR = opj(BASE_DIR, "test_xmls")
TEST_OUTPUT_FILE = opj(ANN_DIR, "test.json")

In [15]:
# Initialize COCO structure
coco = {
    "images": [],
    "annotations": [],
    "categories": []
}

In [16]:
# Initialize category mapping
category_mapping = {
    "tooth": 1,
    "cavity": 1
}

In [17]:
# Add the combined category to COCO categories
coco['categories'].append({
    "id": 1,
    "name": "tooth",
    "supercategory": "none"
})

In [18]:
# Counters for image and annotation IDs
image_id = 0
annotation_id = 0

In [19]:
# Parse each XML file
for xml_file in os.listdir(TEST_ORG_ANN_DIR):
    if not xml_file.endswith('.xml'):
        continue
    
    tree = ET.parse(os.path.join(TEST_ORG_ANN_DIR, xml_file))
    root = tree.getroot()
    
    # Extract image details
    filename = root.find('filename').text.replace(".jpg.jpg", ".jpg")
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    
    image_id += 1
    coco['images'].append({
        "file_name": filename,
        "height": height,
        "width": width,
        "id": image_id
    })
    
    # Extract annotations
    for obj in root.findall('object'):
        category_name = obj.find('name').text
        if category_name not in category_mapping:
            continue
        
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)
        
        o_width = xmax - xmin
        o_height = ymax - ymin
        
        annotation_id += 1
        coco['annotations'].append({
            "segmentation": [],
            "area": o_width * o_height,
            "iscrowd": 0,
            "image_id": image_id,
            "bbox": [xmin, ymin, o_width, o_height],
            "category_id": category_mapping[category_name],
            "id": annotation_id
        })

FileNotFoundError: [Errno 2] No such file or directory: '/home/dmsai2/mmdetection/data/tooth/test_xmls'

In [9]:
coco['categories']

[{'id': 1, 'name': 'tooth', 'supercategory': 'none'}]

In [10]:
coco['annotations'][0]

{'segmentation': [],
 'area': 2365,
 'iscrowd': 0,
 'image_id': 1,
 'bbox': [8, 49, 43, 55],
 'category_id': 1,
 'id': 1}

In [11]:
coco['images'][0]

{'file_name': '1.jpg', 'height': 224, 'width': 224, 'id': 1}

In [12]:
print(len(coco['images']))

17


### Save to JSON

In [13]:
# Save to COCO JSON
with open(VAL_OUTPUT_FILE, 'w') as f:
    json.dump(coco, f, indent=4)